In [1]:
%matplotlib inline

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy as sp
import seaborn as sns

sns.set(context='notebook', font_scale=1.2, rc={'figure.figsize': (12, 5)})
plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])

RANDOM_SEED = 8927
np.random.seed(286)

# Helper function
def stdz(series: pd.Series):
    """Standardize the given pandas Series"""
    return (series - series.mean())/series.std()

### 13E1.
*Add to the following model varying slopes on the predictor $x$:*

$y_{i} \sim Normal(\mu_{i}, \sigma)$

$\mu_{i} = \alpha_{GROUP[i]} + \beta x_{i}$

$\alpha_{GROUP} \sim Normal(\alpha, \sigma_{\alpha})$

$\alpha \sim Normal(0, 10)$

$\beta \sim Normal(0, 1)$

$\sigma \sim HalfCauchy(0, 2)$

$\sigma_{\alpha} \sim HalfCauchy(0, 2)$

Let's do it! To add a varying slope on x,we have to add a dimension to the adaptive prior. This will mean that the $\beta$ parameter becomes the average slope, and then we’ll need a new standard deviation parameter for the slopes and a correlation parameter to estimate the correlation between intercepts and slopes. You can find an annotated example on page 407. Keep in mind that the precise notation varies among statisticians: sometimes (as below) vectors are in square brackets and matrices in parentheses, but others mix and match otherwise, or always use one or the other. As long as it is clear in context, it’ll be fine. And if anyone gives you grief about your notation, just remind (or inform) them that notational conventions vary and ask them which part requires clarification.

$y_{i} \sim Normal(\mu_{i}, \sigma)$

$\mu_{i} = \alpha_{GROUP[i]} + \beta_{GROUP[i]} x_{i}$

$\alpha_{GROUP} \sim Normal(\alpha, \sigma_{\alpha})$

$\beta_{GROUP} \sim Normal(\beta, \sigma_{\beta})$

$\begin{bmatrix} \alpha_{GROUP} \\ \beta_{GROUP} \end{bmatrix} \sim MvNormal(\begin{bmatrix} \alpha \\ \beta \end{bmatrix}, S)$

$S = \begin{pmatrix} \sigma_{\alpha} & 0 \\ 0 & \sigma_{\beta} \end{pmatrix}
     R
     \begin{pmatrix} \sigma_{\alpha} & 0 \\ 0 & \sigma_{\beta} \end{pmatrix}$

$\alpha \sim Normal(0, 10)$

$\beta \sim Normal(0, 10)$

$\sigma_{\alpha} \sim HalfCauchy(0, 1)$

$\sigma_{\beta} \sim HalfCauchy(0, 1)$

$\sigma \sim HalfCauchy(0, 1)$

$R \sim LKJCorr(2)$

If you used different priors for $\beta$ and $\sigma_{\beta}$ and $R$, that’s fine. Just be sure your choices make sense to you.

### 13E2.
*Think up a context in which varying intercepts will be positively correlated with varying slopes. Provide a mechanistic explanation for the correlation.*

This is an open, imaginative problem. So instead of providing a precise answer, let me provide the structure that is being asked for. Then I’ll follow it with some brief examples.

When intercepts and slopes are positively correlated, it means that clusters in the data that have high baselines also have stronger positive associations with a predictor variable. This is merely descriptive. So the mechanistic explanations that are consistent with it are highly diverse.

A very common example comes from educational testing. In this context, clusters are individual schools and observations are individual student test scores. Schools with high average test scores also tend to show greater differences (a larger slope) between poor and rich students within the school.

In growth data, large individuals also tend to grow faster. So for any interval of measurement, the repeat measures of size for individuals show a positive correlation between beginning height (intercept) and the slope across time.

In financial data, for very similar reasons, large investments tend to grow faster. So again, intercepts tend to be positively associated with slopes.

### 13E3.
*When is it possible for a varying slopes model to have fewer effective parameters (as estimated by WAIC or DIC) than the corresponding model with fixed (unpooled) slopes? Explain.*

It is possible for a varying effects model to actually have fewer effective parameters than a corresponding fixed effect model when there is very little variation among clusters. This will create strong shrinkage of the estimates, constraining the individual varying effect parameters. So even though the varying effects model must have more actual parameters in the posterior distribution, it can be less flexible in  fitting the data, because it adaptively regularizes. There’s really nothing special about varying slopes in this respect. Varying intercepts work the same way.

Here’s an example. To answer this problem, you did not need to provide a computational example. I’m just going to provide one for additional clarity. Let’s simulate some data in which there are clusters, but they aren’t very different from one another. For the sake of comprehension, let’s suppose the clusters are individuals and the observations are test scores. Each individual will have a unique "ability" that influences each test score. Our goal in estimation will be to recover these abilities.

In [2]:
N_individuals = 100
N_scores_per_individual = 10

# simulate abilities
ability = np.random.normal(loc=0., scale=0.1, size=N_individuals)

# simulate observed test scores
# sigma here large relative to sigma of ability
N = N_scores_per_individual * N_individuals
stud_id = np.repeat(range(N_individuals), repeats=N_scores_per_individual)
scores = np.random.normal(loc=ability[stud_id], scale=1., size=N).round(2)

# put observable variables in a data frame
d = pd.DataFrame(data=scores, index=stud_id, columns=["score"])
d.head()

,score
0,-1.96
0,0.47
0,-0.06
0,0.39
0,-1.46


Now let’s fit a fixed effect model. This is the "unpooled" model with an intercept for each individual, but with a fixed prior.

In [3]:
with pm.Model() as fixed:
    a_stud = pm.Normal('a_stud', 0., 10., shape=N_individuals)
    mu = a_stud[stud_id]
    sigma = pm.Exponential("sigma", 1.)
    
    score = pm.Normal("score", mu, sigma, observed=d.values)
    
    t_fixed = pm.sample(1000, tune=2000, cores=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [a_stud, sigma]
Sampling 2 chains:   2%|▏         | 106/6000 [02:23<6:11:22,  3.78s/draws]


ValueError: Not enough samples to build a trace.

And this is the corresponding “partial pooling” model with an adaptive prior, the varying effects model. I’m going to use the non-centered (page 419, see Overthinking box on page 422) parameterization here, because this is exactly a common situation in which it is needed to efficiently sample.

In [4]:
with pm.Model() as varying:
    a = pm.Normal("a", 0., 10.)
    sigma_stud = pm.Exponential("sigma_stud", 1.)
    z_stud = pm.Normal('z_stud', 0., 1., shape=N_individuals)
    
    mu = a + sigma_stud*z_stud[stud_id]
    sigma = pm.Exponential("sigma", 1.)
    
    score = pm.Normal("score", mu, sigma, observed=d.values)
    
    t_varying = pm.sample(1000, tune=2000, cores=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [z_stud, sigma_stud, a, sigma]
Sampling 2 chains:   2%|▏         | 141/6000 [03:50<14:17:43,  8.78s/draws]


ValueError: Not enough samples to build a trace.